In [2]:
import json
import pandas as pd
  
df = pd.read_json("/Users/oceane/Desktop/plan-your-trip-with-kayak/coordinates.json")

df[['longitude','latitude']] = df.coordinates.apply(lambda x: pd.Series(str(x).split(",")))
df = df.drop(columns=['coordinates'])


In [3]:
df['longitude'] = df['longitude'].astype(float)
df['latitude'] = df['latitude'].astype(float)

In [4]:
import requests
weather = []
for index, row in df.iterrows():
    appid = '7e208a67c297443752c742b0cdd73fdf'
    lat= row['latitude']
    lon= row['longitude']
    city = row['city']
    # URL = 'https://api.openweathermap.org/data/3.0/onecall'
    # PARAMS = {'lat':lat, 'lon':lon, 'appid': appid, 'units': 'metrics', 'cnt':7, 'exclude':'urrent,minutely,hourly,alerts'}
    # r = requests.get(url = URL, params=PARAMS).json()
    URL = f'https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&units=metric&exclude=current,minutely,hourly,alerts&appid={appid}'
    r = requests.get(URL).json()
    for i in range(7):
        weather.append({'city':city,'lat':lat, 'lon':lon, 'day': r['daily'][i]['dt'], 'temp':r['daily'][i]['temp']['day']})

    

In [5]:

with open('weather.json', 'w') as f:
    f.write(json.dumps(weather, indent=2))


In [6]:
weather_data = pd.read_json("weather.json")
weather_data.head(7)

,city,lat,lon,day,temp
0,Mont Saint-Michel,48.635954,-1.51146,1667559600,13.29
1,Mont Saint-Michel,48.635954,-1.51146,1667646000,12.74
2,Mont Saint-Michel,48.635954,-1.51146,1667732400,14.98
3,Mont Saint-Michel,48.635954,-1.51146,1667818800,16.20
4,Mont Saint-Michel,48.635954,-1.51146,1667905200,15.00
5,Mont Saint-Michel,48.635954,-1.51146,1667991600,15.41
6,Mont Saint-Michel,48.635954,-1.51146,1668078000,16.20


In [7]:
weather_data['temp_mean'] = weather_data['temp'].groupby(weather_data['lat']).transform('mean')



In [8]:
weather_data = weather_data.drop(columns=['temp'])
weather_data = weather_data.drop(columns=['day'])
weather_data.round({'temp_mean': 1})

,city,lat,lon,temp_mean
0,Mont Saint-Michel,48.635954,-1.511460,14.8
1,Mont Saint-Michel,48.635954,-1.511460,14.8
2,Mont Saint-Michel,48.635954,-1.511460,14.8
3,Mont Saint-Michel,48.635954,-1.511460,14.8
4,Mont Saint-Michel,48.635954,-1.511460,14.8
...,...,...,...,...
240,La Rochelle,46.159113,-1.152043,15.9
241,La Rochelle,46.159113,-1.152043,15.9
242,La Rochelle,46.159113,-1.152043,15.9
243,La Rochelle,46.159113,-1.152043,15.9


In [9]:
weather_data = weather_data.drop_duplicates(subset=['lat', 'lon'])
weather_data = weather_data.reset_index(drop=True)

In [10]:
weather_data

,city,lat,lon,temp_mean
0,Mont Saint-Michel,48.635954,-1.511460,14.831429
1,Saint-Malo,48.649518,-2.026041,14.950000
2,Bayeux,49.276462,-0.702474,14.222857
3,Le Havre,49.493898,0.107973,13.990000
4,Rouen,49.440459,1.093966,13.182857
5,Paris,48.858890,2.320041,13.061429
6,Amiens,49.894171,2.295695,12.811429
7,Lille,50.636565,3.063528,12.574286
8,Strasbourg,48.584614,7.750713,12.835714
9,Ch\u00e2teau du Haut-K\u0153nigsbourg,48.249490,7.344296,10.095714


In [11]:
weather_data = weather_data.sort_values(by='temp_mean', ascending=False)

In [12]:
top_five_weather = weather_data.head(5)

In [13]:
top_five_weather.round({'temp_mean': 1})

,city,lat,lon,temp_mean
19,Cassis,43.214036,5.539632,18.4
20,Marseille,43.296174,5.369953,18.1
18,Bormes-les-Mimosas,43.150697,6.341928,17.9
27,Collioure,42.525050,3.083155,17.9
33,Bayonne,43.494514,-1.473666,17.6


In [14]:
import plotly.graph_objects as go
import plotly.express as px

fig = px.scatter_mapbox(top_five_weather, lat="lat", lon="lon", color="temp_mean", size="temp_mean", hover_name="city", zoom=6, mapbox_style="carto-positron")
fig.show()

In [15]:
top_city_names = top_five_weather['city']
top_city_names.to_string('top_city_names.txt',index = False)

In [16]:
top_city_names

19                Cassis
20             Marseille
18    Bormes-les-Mimosas
27             Collioure
33               Bayonne
Name: city, dtype: object

In [18]:
top_city_names.to_csv('top_5_city_names.csv', index=False) 